# Visualising Eye Movement

In [5]:
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib
matplotlib.use("Agg") 

In [6]:
import sys
import os

sys.path.append(os.path.abspath("../src"))  # Add folder_a to sys.path
# from preprocessing import process_line, add_stimulus_events
from config import *

In [8]:
experiment = "EVIL_BASTARD"
events_df = pd.read_parquet(f"{PREPROCESSED_DIR}/{experiment}_events.pq")
samples_df = pd.read_parquet(f"{RAW_DIR}/{experiment}_samples.pq")

In [ ]:
fixpoints = events_df[events_df["event"]=="FIXPOINT"].loc[:,["trial_id", "time", "event", "stimulus_x", "stimulus_y"]]

In [ ]:
events_trials = events_df[(events_df["event"]=="START") | (events_df["event"]=="END")].loc[:,["trial_id", "time", "event"]]

In [45]:
trials = events_trials.pivot(index="trial_id",columns="event")
trials.columns = ["end_time", "start_time"]

In [46]:
trials

,end_time,start_time
trial_id,,
0,450395.0,435248.0
1,470132.0,454960.0
2,489980.0,474836.0
3,510314.0,495158.0


In [ ]:
samples_df

In [48]:
samples["trial_id"]= ""
for t_id, (start_time, end_time) in enumerate(zip(trials["start_time"], trials["end_time"])):
    samples.loc[(samples["time"] >= start_time) & (samples["time"] <= end_time),"trial_id"] = t_id

In [49]:
fixpoints

,trial_id,event,stimulus_x,stimulus_y
4,0,FIXPOINT,981.0,524.0
77,1,FIXPOINT,1019.0,520.0
154,2,FIXPOINT,1000.0,600.0
357,3,FIXPOINT,931.0,522.0


In [108]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation

x = samples.loc[samples["trial_id"]==0]
x_sampled = x[x.index % 100 ==0]

fixpoint = fixpoints.loc[fixpoints["trial_id"]==0]

# Apply Seaborn style
sns.set(style="whitegrid")

fig, ax = plt.subplots(1, 1, figsize = (6, 6))

# Initialize the scatter plot object
scatter_right = ax.scatter([], [], cmap='viridis', label="Right", alpha=0.6)  # 's' is for size, 'c' is for color
scatter_left = ax.scatter([], [], cmap='coolwarm', label="Left", alpha=0.6)  # 's' is for size, 'c' is for color
scatter_fixpoint = ax.scatter([], [], c="red", label="Fixpoint", alpha=0.6, s=50)  # 's' is for size, 'c' is for color

x_min = min(min(x_sampled["x_right"]), min(x_sampled["x_left"]))
x_max = max(max(x_sampled["x_right"]), max(x_sampled["x_left"]))
y_min =  min(min(x_sampled["y_right"]), min(x_sampled["y_left"]))
y_max = max(max(x_sampled["y_right"]), max(x_sampled["y_left"]))
ax.set_xlim([x_min, x_max]) # fix the x axis
ax.set_ylim([y_min, y_max]) # fix the y axis

# Add legend to distinguish between the three points
ax.legend()

# Initialize function: this is called to create the background of each frame
def init():
    scatter_right.set_offsets([])  # Set empty offsets initially
    scatter_left.set_offsets([])  # Set empty offsets initially
    scatter_fixpoint.set_offsets([])  # Set empty offsets initially    
    # scatter.set_sizes([])    # Set empty sizes initially
    # scatter.set_array([])    # Set empty color array initially
    return scatter_left, scatter_right, scatter_fixpoint,

def animate(i):
    # Update the scatter plot data for each frame
    coords_right = np.c_[x_sampled["x_left"].iloc[i], x_sampled["y_right"].iloc[i]]  # Update coordinates
    # sizes = x_sampled["x_velocity_right"][:i]                   # Update sizes based on density
    # colors = x_sampled["x_velocity_right"][:i]                   # Update color if needed (optional)

    coords_left = np.c_[x_sampled["x_left"].iloc[i], x_sampled["y_left"].iloc[i]]  # Update coordinates

    if x_sampled["time"].iloc[i] > fixpoint["time"].iloc[0]:
        coords_fixpoint = np.array([[fixpoint["stimulus_x"].iloc[0], fixpoint["stimulus_y"].iloc[0]]])
    else:
        coords_fixpoint = np.array([[-1, -1]]) 

    # Update the scatter plot data for each frame
    scatter_right.set_offsets(coords_right)  # Update coordinates
    # scatter.set_sizes(sizes)                    # Update sizes based on density
    # scatter.set_array(colors)                    # Update color if needed (optional)
    
    scatter_left.set_offsets(coords_left)
    scatter_fixpoint.set_offsets(coords_fixpoint)

    return scatter_right, scatter_left, scatter_fixpoint

anim = animation.FuncAnimation(
    fig, 
    animate, 
    frames = len(x_sampled), 
    interval=100,
    blit = False
)

anim.save("animation.mp4", writer="ffmpeg", fps=30)


/tmp/ipykernel_3477/227582175.py:16: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  scatter_right = ax.scatter([], [], cmap='viridis', label="Right", alpha=0.6)  # 's' is for size, 'c' is for color
/tmp/ipykernel_3477/227582175.py:17: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  scatter_left = ax.scatter([], [], cmap='coolwarm', label="Left", alpha=0.6)  # 's' is for size, 'c' is for color
